In [1]:
# ==============================================================
# CNN Lite (Colab-ready): bajo consumo de RAM
# - Modelos: LeNet5 (desde cero), MobileNetV2, EfficientNetB0 (transfer)
# - Dataset: CIFAR-10 (32x32x3), resize on-the-fly a 128x128
# - Sin copias gigantes en memoria, batch pequeño, mixed precision opcional
# ==============================================================

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np, gc, os

# ----------------------------
# 0) CONFIG (EDITA AQUÍ)
# ----------------------------
CONFIG = {
    "model_name": "MobileNetV2",   # "LeNet5", "MobileNetV2", "EfficientNetB0"
    "batch_size": 32,
    "epochs": 5,
    "learning_rate": 1e-3,
    "input_size": (128, 128),      # Reduce a (96, 96) si sigues corto de RAM
    "val_split": 0.1,
    "data_augmentation": False,    # True si te alcanza RAM
    "use_mixed_precision": True,   # mixed_float16 (ahorra memoria en GPU)
    "seed": 42,
}

# Mixed precision (si hay GPU, suele ahorrar RAM y acelerar)
if CONFIG["use_mixed_precision"]:
    try:
        from tensorflow.keras import mixed_precision
        mixed_precision.set_global_policy("mixed_float16")
    except Exception as e:
        print("No se activó mixed precision:", e)

# ----------------------------
# 1) CARGA CIFAR-10 (pequeño)
# ----------------------------
NUM_CLASSES = 10
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
y_train = y_train.flatten()
y_test  = y_test.flatten()

# Split sin copiar grandes arrays (take/skip sobre tf.data)
total = x_train.shape[0]
val_count = int(total * CONFIG["val_split"])

# ----------------------------
# 2) PIPELINE tf.data (resize on-the-fly)
# ----------------------------
AUTO = tf.data.AUTOTUNE
BATCH = CONFIG["batch_size"]
INPUT_SIZE = CONFIG["input_size"]

def preprocess_img(img, label):
    img = tf.image.resize(img, INPUT_SIZE)
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

if CONFIG["data_augmentation"]:
    aug = keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.05),
        layers.RandomZoom(0.1),
    ])
    def with_aug(img, label):
        img, label = preprocess_img(img, label)
        img = aug(img, training=True)
        return img, label
else:
    with_aug = preprocess_img

train_ds_full = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_ds = train_ds_full.take(val_count).map(preprocess_img, num_parallel_calls=AUTO)
train_ds = train_ds_full.skip(val_count).map(with_aug, num_parallel_calls=AUTO)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).map(preprocess_img, num_parallel_calls=AUTO)

train_ds = train_ds.shuffle(2048, seed=CONFIG["seed"]).batch(BATCH).prefetch(AUTO)
val_ds   = val_ds.batch(BATCH).prefetch(AUTO)
test_ds  = test_ds.batch(BATCH).prefetch(AUTO)

# ----------------------------
# 3) MODELOS
# ----------------------------
def explain_model(name: str) -> str:
    n = name.lower()
    if n == "lenet5":
        return ("LeNet-5: pionera (Conv+Pool apilados → FC). "
                "Extrae bordes/patrones simples y reduce dimensionalidad con pooling.")
    if n == "mobilenetv2":
        return ("MobileNetV2: depthwise separable + inverted residuals. "
                "Diseñada para dispositivos móviles: muy liviana y eficiente.")
    if n == "efficientnetb0":
        return ("EfficientNetB0: escalado compuesto (ancho, profundidad, resolución) "
                "con bloques MBConv + squeeze-and-excitation. Excelente relación precisión/costo.")
    return "Modelo no documentado."

def build_lenet5(input_shape=(128,128,3), num_classes=10):
    inputs = keras.Input(shape=input_shape)
    x = layers.Conv2D(6, (5,5), padding="same", activation="relu")(inputs)
    x = layers.AveragePooling2D()(x)
    x = layers.Conv2D(16, (5,5), padding="same", activation="relu")(x)
    x = layers.AveragePooling2D()(x)
    x = layers.Conv2D(120, (5,5), padding="same", activation="relu")(x)
    x = layers.Flatten()(x)
    x = layers.Dense(84, activation="relu")(x)
    outputs = layers.Dense(num_classes, activation="softmax", dtype="float32")(x)  # salidas en float32
    return keras.Model(inputs, outputs, name="LeNet5")

def build_transfer(name, input_shape=(128,128,3), num_classes=10, freeze=True):
    if name == "MobileNetV2":
        from tensorflow.keras.applications import MobileNetV2
        from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
        Base = MobileNetV2; preprocess = preprocess_input
    elif name == "EfficientNetB0":
        from tensorflow.keras.applications import EfficientNetB0
        from tensorflow.keras.applications.efficientnet import preprocess_input
        Base = EfficientNetB0; preprocess = preprocess_input
    else:
        raise ValueError("Solo MobileNetV2 o EfficientNetB0 en modo Lite.")

    inp = keras.Input(shape=input_shape)
    x = layers.Lambda(preprocess, name="preprocess")(inp)
    base = Base(include_top=False, weights="imagenet", input_tensor=x)
    if freeze:
        for l in base.layers:
            l.trainable = False
    x = layers.GlobalAveragePooling2D()(base.output)
    x = layers.Dropout(0.2)(x)
    out = layers.Dense(num_classes, activation="softmax", dtype="float32")(x)
    return keras.Model(inp, out, name=f"{name}_lite")

input_shape = (*INPUT_SIZE, 3)
name = CONFIG["model_name"]

keras.backend.clear_session(); gc.collect()

if name == "LeNet5":
    model = build_lenet5(input_shape, NUM_CLASSES)
else:
    model = build_transfer(name, input_shape, NUM_CLASSES, freeze=True)

opt = keras.optimizers.Adam(learning_rate=CONFIG["learning_rate"])
model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

print("\n== ARQUITECTURA =="); model.summary()
print("\n== EXPLICACIÓN =="); print(explain_model(name))

callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True)
]

history = model.fit(train_ds, validation_data=val_ds, epochs=CONFIG["epochs"], callbacks=callbacks, verbose=1)

print("\n== EVALUACIÓN TEST ==")
test_loss, test_acc = model.evaluate(test_ds, verbose=0)
print(f"Test accuracy: {test_acc:.4f} | Test loss: {test_loss:.4f}")

# Reporte compacto
from sklearn.metrics import classification_report, confusion_matrix
y_true, y_pred = [], []
for xb, yb in test_ds:
    probs = model.predict(xb, verbose=0)
    y_true.extend(yb.numpy().tolist())
    y_pred.extend(np.argmax(probs, axis=1).tolist())

print("\n== CLASSIFICATION REPORT ==")
print(classification_report(y_true, y_pred, digits=4))
print("\n== CONFUSION MATRIX ==")
print(confusion_matrix(y_true, y_pred))


/tmp/ipython-input-3659807033.py:125: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base = Base(include_top=False, weights="imagenet", input_tensor=x)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

== ARQUITECTURA ==


Model: "MobileNetV2_lite"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ preprocess (Lambda) │ (None, 128, 128,  │          0 │ input_layer[0][0] │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 64, 64,    │        864 │ preprocess[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 64, 64,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 64, 64,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 64, 64,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 64, 64,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 64, 64,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 65, 65,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 32, 32,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │          0 │ block_1_depthwis

 Total params: 2,270,794 (8.66 MB)

 Trainable params: 12,810 (50.04 KB)

 Non-trainable params: 2,257,984 (8.61 MB)


== EXPLICACIÓN ==
MobileNetV2: depthwise separable + inverted residuals. Diseñada para dispositivos móviles: muy liviana y eficiente.
Epoch 1/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 346s 244ms/step - accuracy: 0.1126 - loss: 2.3005 - val_accuracy: 0.1766 - val_loss: 2.2690
Epoch 2/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 389s 248ms/step - accuracy: 0.1641 - loss: 2.2645 - val_accuracy: 0.2164 - val_loss: 2.2422
Epoch 3/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 365s 236ms/step - accuracy: 0.1941 - loss: 2.2365 - val_accuracy: 0.2440 - val_loss: 2.2115
Epoch 4/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 377s 233ms/step - accuracy: 0.2029 - loss: 2.2151 - val_accuracy: 0.2400 - val_loss: 2.1937
Epoch 5/5
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 384s 234ms/step - accuracy: 0.2124 - loss: 2.1984 - val_accuracy: 0.2756 - val_loss: 2.1698

== EVALUACIÓN TEST ==
Test accuracy: 0.2701 | Test loss: 2.1709

== CLASSIFICATION REPORT ==
              precision    recall  f1-score   support

           0     0.2230    0.4090    0.2886      10

In [ ]:
# ==============================================================
# CNN Combo (Keras/TensorFlow, Colab-ready) con Transfer Learning
# - Un solo script para entrenar varias arquitecturas clásicas
# - Dataset: CIFAR-10 por defecto (10 clases, 32x32)
# - Cambia CONFIG["model_name"] para probar distintos modelos
# ==============================================================

import os, sys, math, tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

# ----------------------------
# 0) CONFIGURACIÓN (EDITA AQUÍ)
# ----------------------------
CONFIG = {
    "model_name": "ResNet50",   # "LeNet5","VGG16","ResNet50","InceptionV3","Xception","MobileNetV2","EfficientNetB0"
    "dataset": "cifar10",       # "cifar10" o "fashion_mnist" (se autoajusta a 3 canales)
    "batch_size": 128,
    "epochs": 5,                # Sube a 20+ si quieres mejor accuracy
    "learning_rate": 1e-3,
    "data_augmentation": True,
    "freeze_base": True,        # Congela el feature extractor pre-entrenado
    "fine_tune_at": None,       # Si no es None, descongela capas desde este índice
    "val_split": 0.1,
    "seed": 42,
}

# ---------------------------------------------------
# 1) UTILIDADES: explicación resumida de cada modelo
# ---------------------------------------------------
def explain_model(name: str) -> str:
    name = name.lower()
    if name == "lenet5":
        return (
            "LeNet-5 (1998): arquitectura pionera para dígitos manuscritos.\n"
            "- Bloques: Conv(5x5) + Pooling + Conv(5x5) + Pooling + FC.\n"
            "- Idea clave: extraer bordes y patrones simples con filtros pequeños y reducir dimensionalidad con pooling."
        )
    if name == "vgg16":
        return (
            "VGG16 (2014): muchas convoluciones 3x3 apiladas.\n"
            "- Bloques: [Conv3x3,Conv3x3]+MaxPool repetidos, luego FC.\n"
            "- Idea clave: profundidad + filtros pequeños = mejor representación con arquitectura simple y modular."
        )
    if name == "resnet50":
        return (
            "ResNet50 (2015): 'skip connections' o conexiones residuales.\n"
            "- Bloques: bottlenecks con atajos (identity/conv) que saltan capas.\n"
            "- Idea clave: el gradiente fluye por los atajos y permite redes muy profundas."
        )
    if name == "inceptionv3":
        return (
            "InceptionV3 (2015): módulos con ramas paralelas (1x1,3x3,5x5) + pooling.\n"
            "- Bloques: Inception modules que mezclan escalas de forma eficiente.\n"
            "- Idea clave: capturar patrones en múltiples escalas sin disparar parámetros."
        )
    if name == "xception":
        return (
            "Xception (2017): 'depthwise separable convolutions' (separa espacial vs canales).\n"
            "- Bloques: Depthwise (por canal) + Pointwise(1x1) para mezclar canales.\n"
            "- Idea clave: más eficiencia computacional y mejor rendimiento."
        )
    if name == "mobilenetv2":
        return (
            "MobileNetV2 (2018): separables + 'inverted residuals' con proyección lineal.\n"
            "- Bloques: expand (1x1) -> depthwise (3x3) -> project (1x1), con atajos.\n"
            "- Idea clave: modelos ligeros para móviles manteniendo precisión decente."
        )
    if name == "efficientnetb0":
        return (
            "EfficientNetB0 (2019): escalado compuesto ancho+profundidad+resolución.\n"
            "- Bloques: MBConv (como MobileNet) con squeeze-and-excitation.\n"
            "- Idea clave: buscar automáticamente la mejor combinación de escalado para máxima eficiencia."
        )
    return "Modelo no documentado."

# ---------------------------------------------------
# 2) CARGA DE DATOS (CIFAR-10 o Fashion-MNIST)
#    * CIFAR-10 ya viene en 3 canales (RGB, 32x32)
#    * Fashion-MNIST es 1 canal (28x28) → se repite a 3 canales
# ---------------------------------------------------
NUM_CLASSES = 10

def load_dataset(name):
    if name == "cifar10":
        (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
        channel_3 = True
    elif name == "fashion_mnist":
        (x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
        # Expandimos a [H,W,1] y repetimos canal para simular RGB
        x_train = x_train[..., None]
        x_test  = x_test[..., None]
        x_train = tf.repeat(x_train, repeats=3, axis=-1).numpy()
        x_test  = tf.repeat(x_test,  repeats=3, axis=-1).numpy()
        channel_3 = True
    else:
        raise ValueError("Dataset no soportado.")
    y_train = y_train.flatten()
    y_test  = y_test.flatten()
    return (x_train, y_train), (x_test, y_test), channel_3

(x_train, y_train), (x_test, y_test), _ = load_dataset(CONFIG["dataset"])

# ---------------------------------------------------
# 3) ELEGIR TAMAÑO DE ENTRADA SEGÚN ARQUITECTURA
#    (algunos modelos esperan 299x299)
# ---------------------------------------------------
if CONFIG["model_name"] in ["InceptionV3", "Xception"]:
    INPUT_SIZE = (299, 299)
else:
    INPUT_SIZE = (224, 224)

# Normalización a [0,1] y resize
def preprocess_images(x, target_size):
    x = x.astype("float32") / 255.0
    return tf.image.resize(x, target_size)

x_train = preprocess_images(x_train, INPUT_SIZE)
x_test  = preprocess_images(x_test,  INPUT_SIZE)

# One-hot opcional (Keras acepta enteros en sparse_categorical_crossentropy)
# y_train_cat = to_categorical(y_train, NUM_CLASSES)
# y_test_cat  = to_categorical(y_test, NUM_CLASSES)

# ---------------------------------------------------
# 4) DATASETS TF + AUGMENTATION opcional
# ---------------------------------------------------
AUTO = tf.data.AUTOTUNE
BATCH = CONFIG["batch_size"]
VAL_SPLIT = CONFIG["val_split"]

total = x_train.shape[0]
val_count = int(total * VAL_SPLIT)
x_val, y_val = x_train[:val_count], y_train[:val_count]
x_train, y_train = x_train[val_count:], y_train[val_count:]

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val_ds   = tf.data.Dataset.from_tensor_slices((x_val, y_val))
test_ds  = tf.data.Dataset.from_tensor_slices((x_test, y_test))

if CONFIG["data_augmentation"]:
    aug = keras.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.05),
        layers.RandomZoom(0.1),
    ], name="data_augmentation")
else:
    aug = keras.Sequential([], name="no_aug")

def prep(ds, training=False):
    ds = ds.shuffle(2048, seed=CONFIG["seed"]) if training else ds
    ds = ds.batch(BATCH).prefetch(AUTO)
    if training and CONFIG["data_augmentation"]:
        # Aplicamos augmentation en el pipeline
        ds = ds.map(lambda x, y: (aug(x, training=True), y), num_parallel_calls=AUTO)
    return ds

train_ds = prep(train_ds, training=True)
val_ds   = prep(val_ds, training=False)
test_ds  = prep(test_ds, training=False)

# ---------------------------------------------------
# 5) CONSTRUCTOR DE MODELOS
#    - LeNet5: desde cero para docencia
#    - Resto: aplicaciones preentrenadas (ImageNet) + nueva cabeza
# ---------------------------------------------------
def build_lenet5(input_shape=(32,32,3), num_classes=10):
    # Nota: redimensionamos a INPUT_SIZE arriba, pero LeNet se diseñó para 32x32.
    # Aquí adaptamos a INPUT_SIZE con conv "same".
    inputs = keras.Input(shape=input_shape)
    x = layers.Conv2D(6, (5,5), padding="same", activation="relu")(inputs)
    x = layers.AveragePooling2D()(x)
    x = layers.Conv2D(16, (5,5), padding="same", activation="relu")(x)
    x = layers.AveragePooling2D()(x)
    x = layers.Conv2D(120, (5,5), padding="same", activation="relu")(x)
    x = layers.Flatten()(x)
    x = layers.Dense(84, activation="relu")(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = keras.Model(inputs, outputs, name="LeNet5")
    return model

def build_transfer_model(name, input_shape, num_classes=10, freeze_base=True):
    # Capa de preprocesamiento específica de cada familia (opcional)
    preprocess = None
    BaseClass = None

    if name == "VGG16":
        from tensorflow.keras.applications import VGG16
        from tensorflow.keras.applications.vgg16 import preprocess_input
        BaseClass = VGG16; preprocess = preprocess_input
    elif name == "ResNet50":
        from tensorflow.keras.applications import ResNet50
        from tensorflow.keras.applications.resnet import preprocess_input
        BaseClass = ResNet50; preprocess = preprocess_input
    elif name == "InceptionV3":
        from tensorflow.keras.applications import InceptionV3
        from tensorflow.keras.applications.inception_v3 import preprocess_input
        BaseClass = InceptionV3; preprocess = preprocess_input
    elif name == "Xception":
        from tensorflow.keras.applications import Xception
        from tensorflow.keras.applications.xception import preprocess_input
        BaseClass = Xception; preprocess = preprocess_input
    elif name == "MobileNetV2":
        from tensorflow.keras.applications import MobileNetV2
        from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
        BaseClass = MobileNetV2; preprocess = preprocess_input
    elif name == "EfficientNetB0":
        from tensorflow.keras.applications import EfficientNetB0
        from tensorflow.keras.applications.efficientnet import preprocess_input
        BaseClass = EfficientNetB0; preprocess = preprocess_input
    else:
        raise ValueError("Modelo no soportado en transfer.")

    inputs = keras.Input(shape=input_shape)
    x = inputs
    if preprocess is not None:
        # Ajusta canal de color, media y escala según el backbone
        x = layers.Lambda(preprocess, name="preprocess")(x)

    # include_top=False para usar como extractor de características
    base = BaseClass(include_top=False, weights="imagenet", input_tensor=x)
    if freeze_base:
        for layer in base.layers:
            layer.trainable = False

    # Cabeza de clasificación (global pooling + dropout + softmax)
    x = layers.GlobalAveragePooling2D(name="gap")(base.output)
    x = layers.Dropout(0.2, name="dropout")(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)
    model = keras.Model(inputs, outputs, name=f"{name}_transfer")

    return model, base

# Construcción según selección
input_shape = (*INPUT_SIZE, 3)
model_name = CONFIG["model_name"]

if model_name == "LeNet5":
    model = build_lenet5(input_shape=input_shape, num_classes=NUM_CLASSES)
    base = None
else:
    model, base = build_transfer_model(model_name, input_shape, NUM_CLASSES, CONFIG["freeze_base"])

# Fine-tuning opcional: descongelar parte del backbone
if base is not None and CONFIG["fine_tune_at"] is not None:
    # Descongela desde la capa 'fine_tune_at' (índice en la lista base.layers)
    for i, layer in enumerate(base.layers):
        layer.trainable = (i >= CONFIG["fine_tune_at"])

# ---------------------------------------------------
# 6) COMPILACIÓN, ENTRENAMIENTO Y EVALUACIÓN
# ---------------------------------------------------
opt = keras.optimizers.Adam(learning_rate=CONFIG["learning_rate"])
model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"])

print("\n================= ARQUITECTURA =================")
print(model.summary())

print("\n================= EXPLICACIÓN ==================")
print(explain_model(model_name))

callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=3, restore_best_weights=True)
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=CONFIG["epochs"],
    callbacks=callbacks,
    verbose=1
)

print("\n================= EVALUACIÓN TEST ===============")
test_loss, test_acc = model.evaluate(test_ds, verbose=0)
print(f"Test accuracy: {test_acc:.4f} | Test loss: {test_loss:.4f}")

# ---------------------------------------------------
# 7) PREDICCIONES Y MATRIZ DE CONFUSIÓN (opcional)
# ---------------------------------------------------
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

y_true = []
y_pred = []

for xb, yb in test_ds:
    probs = model.predict(xb, verbose=0)
    y_true.extend(yb.numpy().tolist())
    y_pred.extend(np.argmax(probs, axis=1).tolist())

print("\n=============== CLASSIFICATION REPORT ===========")
print(classification_report(y_true, y_pred, digits=4))

print("\n================== CONFUSION MATRIX =============")
print(confusion_matrix(y_true, y_pred))
